# Cloud Workshop Azure Databricks
## 13. Azure Databricks et ML Flow - Batch
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.3 12/06/2019

## 1. Review the experiment

#### Choisir un run MLFlow :

In [4]:
run_id1 = "ddf75cd3008f4075ad1a429888b410e4"
model_uri = "runs:/" + run_id1 + "/model"

## 2. Load MLflow Model as a scikit-learn Model
You can use the MLflow API to load the model from the MLflow server that was produced by a given run.
Once you load it, it is a just a scikit-learn model and you can explore it or use it.

In [6]:
import mlflow.sklearn

model = mlflow.sklearn.load_model(model_uri=model_uri)
model.coef_

/databricks/python/lib/python3.5/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator ElasticNet from version 0.19.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
 UserWarning)
 Out[ 2 ]: 
array([ -97.8872525 , -196.74240011, 497.60452471, 284.0129078 ,
 -115.89225934, 0. , -231.83208264, 43.94931738,
 506.77001432, 97.71645591])

In [7]:
from sklearn import datasets
import numpy as np
import pandas as pd

# Load Diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

# Create pandas DataFrame for sklearn ElasticNet linear_model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)

In [8]:
# Get a prediction for a row of the dataset
model.predict(data[0:1].drop(["progression"], axis=1))

Out[ 4 ]: array([ 199.95454016])

In [9]:
# Create a Spark DataFrame from the original pandas DataFrame minus the column you want to predict.
# Use this to simulate what this would be like if you had a big data set e.g. click logs that was 
# regularly being updated that you wanted to score.

dataframe = spark.createDataFrame(data.drop(["progression"], axis=1))

## 3. Use an MLflow Model for Batch Inference
You can get a PySpark UDF to do some batch inference using one of the models.

In [11]:
import mlflow.pyfunc
pyfunc_udf = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

In [12]:
predicted_df = dataframe.withColumn("prediction", pyfunc_udf('age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'))

In [13]:
display(predicted_df)

age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,prediction
0.0380759064334241,0.0506801187398187,0.0616962065186885,0.0218723549949558,-0.0442234984244464,-0.0348207628376986,-0.0434008456520269,-0.00259226199818282,0.0199084208763183,-0.0176461251598052,199.95454016293704
-0.00188201652779104,-0.044641636506989,-0.0514740612388061,-0.0263278347173518,-0.00844872411121698,-0.019163339748222,0.0744115640787594,-0.0394933828740919,-0.0683297436244215,-0.09220404962683,67.53374220060797
0.0852989062966783,0.0506801187398187,0.0444512133365941,-0.00567061055493425,-0.0455994512826475,-0.0341944659141195,-0.0323559322397657,-0.00259226199818282,0.00286377051894013,-0.0259303389894746,167.07994136589085
-0.0890629393522603,-0.044641636506989,-0.0115950145052127,-0.0366564467985606,0.0121905687618,0.0249905933641021,-0.0360375700438527,0.0343088588777263,0.0226920225667445,-0.0093619113301358,173.65759671436638
0.00538306037424807,-0.044641636506989,-0.0363846922044735,0.0218723549949558,0.00393485161259318,0.0155961395104161,0.0081420836051921,-0.00259226199818282,-0.0319914449413559,-0.0466408735636482,126.43804053293003
-0.0926954778032799,-0.044641636506989,-0.0406959404999971,-0.0194420933298793,-0.0689906498720667,-0.0792878444118122,0.0412768238419757,-0.076394503750001,-0.0411803851880079,-0.0963461565416647,110.17208957619022
-0.0454724779400257,0.0506801187398187,-0.0471628129432825,-0.015999222636143,-0.040095639849843,-0.0248000120604336,7.78807997017968E-4,-0.0394933828740919,-0.0629129499162512,-0.0383566597339788,86.87053008755379
0.063503675590561,0.0506801187398187,-0.00189470584028465,0.0666296740135272,0.0906198816792644,0.108914381123697,0.0228686348215404,0.0177033544835672,-0.0358167281015492,0.00306440941436832,122.21922463575306
0.0417084448844436,0.0506801187398187,0.0616962065186885,-0.0400993174922969,-0.0139525355440215,0.00620168565673016,-0.0286742944356786,-0.00259226199818282,-0.0149564750249113,0.0113486232440377,160.24072898975876
-0.0709002470971626,-0.044641636506989,0.0390621529671896,-0.0332135761048244,-0.0125765826858204,-0.034507614375909,-0.0249926566315915,-0.00259226199818282,0.0677363261102861,-0.0135040182449705,219.1750832622509


> Fin